In [15]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores.faiss import FAISS

In [17]:
# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [18]:
# 로컬 파일 저장소 설정
store = LocalFileStore("./cache2/")

In [19]:
# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings = embeddings,
    document_embedding_cache = store,
    namespace=embeddings.model,         # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [20]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

['text-embedding-3-small5308cd1c-e8fe-51d4-a9d2-966426e5e41a',
 'text-embedding-3-smallbc313e6c-0c06-5c61-96b2-ec7268aee48f',
 'text-embedding-3-smalle0ddaaa1-5d36-53fd-86ba-5bdd57306f68']

In [21]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [22]:
# 문서 로드
raw_documents = TextLoader("./data/book_document.txt", encoding='utf-8').load()

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [24]:
documents = text_splitter.split_documents(raw_documents)

In [25]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: total: 0 ns
Wall time: 5.98 ms


In [26]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 0 ns
Wall time: 4.98 ms
